CAPÍTULO 16 - Clustering
-

Os métodos de aprendizado de máquina podem ser classificados em duas principais categorias: aprendizado supervisionado e apredizado não supervisionado

Até agora usamos aprendizado supervisionado pois treinamos com um alvo y ou uma variável resposta ou seja, sabiamos qual a reposta "correta". Já o não supervisionado nós não sabemos qual é a resposta "correta". Muitos desses métodos envolvem clustering (agrupamento), e dois métodos principais: clustering k-means e clustering hierárquico.

16.2 k-means

Usaremos dados sobre vinhos em nosso exemplo de k-means

In [18]:
import pandas as pd

wine = pd.read_csv('../../data/wine.csv')

print(wine.head())

   Cultivar  Alcohol  Malic acid   Ash  Alcalinity of ash    Magnesium  \
0         1    14.23        1.71  2.43                 15.6        127   
1         1    13.20        1.78  2.14                 11.2        100   
2         1    13.16        2.36  2.67                 18.6        101   
3         1    14.37        1.95  2.50                 16.8        113   
4         1    13.24        2.59  2.87                 21.0        118   

   Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   
3           3.85        3.49                  0.24             2.18   
4           2.80        2.69                  0.39             1.82   

   Color intensity   Hue  OD280/OD315 of diluted wines  Proline              
0             5.64  1.04                          

Vamos descartar a coluna cultivar pois ela se relaciona muito de perto com os clusters em nossos dados

In [19]:
wine = wine.drop('Cultivar', axis=1)
print(wine.head())

   Alcohol  Malic acid   Ash  Alcalinity of ash    Magnesium  Total phenols  \
0    14.23        1.71  2.43                 15.6        127           2.80   
1    13.20        1.78  2.14                 11.2        100           2.65   
2    13.16        2.36  2.67                 18.6        101           2.80   
3    14.37        1.95  2.50                 16.8        113           3.85   
4    13.24        2.59  2.87                 21.0        118           2.80   

   Flavanoids  Nonflavanoid phenols  Proanthocyanins  Color intensity   Hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   OD280/OD315 of diluted wines  Proline          

A sklearn têm uma im plementação do algoritimo de k-means chamado KMeans. Nesse exemplo, definiremos k = 3 e usaremos todos os dados de nosso conjunto

In [22]:
from sklearn.cluster import KMeans
import os
os.environ["OMP_NUM_THREADS"] = "1"

#cria 3 clusters
#usa uma semente aleatória igual a 42
#você pode optar por não usar random_state ou pode usar um valor diferente; 42 garantirá que seus resultados sejam iguais àqueles exibidos no livro

kmeans = KMeans(n_clusters=3, random_state=42).fit(wine.values)

c:\Users\User\anaconda3\envs\pfe_certo\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
